In [1]:
from tree import treemodule
from tree import treeutils
import numpy as np

alltrees = treemodule.CTree()
wdir = '/home/hoseung/Work/data/05427/'
alltrees.load(filename= wdir + 'rhalo/Trees/tree_0_0_0.dat')


Loading Consistent Tree data from ASCII is done


In [2]:
def extract_a_tree(alltrees, idx_last):
    i_last = np.where(alltrees['id'] == idx_last)
    return alltrees[np.where(alltrees['tree_root_id'] == idx_last)]
    
def get_progenitors(treedata, idx):
    iprgs = np.where(treedata['desc_id'] == idx)
    idx_prgs = treedata['id'][iprgs]
    return idx_prgs

In [3]:
atree = extract_a_tree(alltrees.data, 110226)

In [4]:
get_progenitors(atree, 107277)

array([], dtype=int64)

In [5]:
nout_max = alltrees.data['nout'].max()
nout_fi = 187
alltrees.data['nout'] += nout_fi - nout_max

#max_nout = nout_fi
i_final = np.where(alltrees.data["nout"] == nout_fi)
ttt_sub = alltrees.data[i_final]
#ttt.data[0]

In [8]:
def link_circle_up(x, y, r, ax, finish=0):
    """
    Given two points, draw circle at the first point and link it to the second point
    without drawing the second point by default (so that it can repeat to build a long thread of bids).
    for the last point, pass the radius of the last circle to the argument 'finish'
    
    For example,

    fig = plt.figure()
    ax = fig.add_subplot(111)
    xpos = [1,1] &  ypos = [2,4]
    link_circle(xpos, ypos, 10, ax)

    xpos = [1,2] & ypos = [4,6]
    link_circle(xpos, ypos, 30, ax, finish=30)
    fig.show()
    """
    ax.plot(x[0], y[0], 'o', ms=r, lw=2,  alpha=0.7, mfc='orange')
    ax.plot(x, y, '-', c='black',alpha=0.7)
    if finish > 0:
        ax.plot(x[1], y[1], 'o', ms=20, lw=2, alpha=0.7, mfc='orange')    

def recursive_tree(idx, tt, nstep, ax, x0, y0, dx, mass_unit=1e10):
    import tree.draw_merger_tree as dmt
    prgs = get_progenitors(tt, idx)
    i_this_gal = np.where(tt['id'] == idx)
    m = np.sqrt(tt[i_this_gal]["mvir"] / mass_unit)
    #print("IDX:", idx, "prgs: ",prgs, "mass:", m, i_this_gal)
    nprg = len(prgs)
    if nstep == 0:
        return 
    else:
        if nprg == 0:
            return
        else:
            if nprg > 1:
                dx *= 0.9
#                print("Branch!", nprg)

            xarr = dmt.get_xarr(nprg) * dx + x0
            for i, x in zip(prgs, xarr):
                link_circle_up([x0, x], [y0, y0 + 1], m, ax)
                recursive_tree(i, tt, nstep - 1, ax, x, y0 + 1, dx, mass_unit=mass_unit)


In [18]:
#iii = np.where(ttt_sub['Orig_halo_id'] == 7)[0]
iii = np.where(ttt_sub['id'] == 110226)[0]

In [19]:
ttt_sub[iii]

array([], 
      dtype=[('aexp', '<f8'), ('id', '<i8'), ('desc_aexp', '<i8'), ('desc_id', '<i8'), ('nprog', '<i8'), ('pid', '<i8'), ('upid', '<i8'), ('desc_pid', '<i8'), ('phantom', '<i8'), ('sam_mvir', '<f8'), ('mvir', '<f8'), ('rvir', '<f8'), ('rs', '<i8'), ('vrms', '<f8'), ('mmp', '<f8'), ('aexp_last_MM', '<f8'), ('vmax', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<i8'), ('jx', '<i8'), ('jy', '<i8'), ('jz', '<f8'), ('spin', '<i8'), ('b_id', '<i8'), ('d_id', '<i8'), ('tree_root_id', '<i8'), ('Orig_halo_id', '<i8'), ('nout', '<i8'), ('next_coprogenitor_d_id', '<i8'), ('last_progenitor_d_id', '<f8')])

In [17]:
import matplotlib.pyplot as plt


galid = 110478

nout_fi = 187
nout_ini = 30

final_gals = ttt_sub['id']
final_gals_org = ttt_sub['Orig_halo_id']

#for galid in final_gals[2:3]:
for galid in [110478]:
    print(galid)
        
    fig = plt.figure(figsize=[6,6])
    plt.ioff()
    ax = fig.add_subplot(111)
    #galid = 6033
    sidgal = str(galid).zfill(5)
    
    nouts = np.unique(range(nout_fi - nout_ini + 2))
    zreds = np.unique(alltrees.data["aexp"])[:len(nouts)]
    zreds = ["%.2f" % i for i in zreds]
    #print(zreds)
    atree = extract_a_tree(alltrees.data, galid)
    recursive_tree(galid, atree, 120, ax, 0, 0, 1, mass_unit=1e8)
    
    # y axis label (redshift)
    ax.set_ylabel("Redshift")
    ax.set_xlim([-3,1])
    ax.set_ylim([0,151])
    plt.yticks(nouts[1:151:10], zreds[1:151:10])
    ax.set_title(sidgal)
    #fig.show()
    plt.savefig(wdir + "mergertrees/" + sidgal + '.png')
    plt.close()

110478


In [57]:
ttt.data.dtype

dtype([('aexp', '<f8'), ('id', '<i8'), ('desc_aexp', '<i8'), ('desc_id', '<i8'), ('nprog', '<i8'), ('pid', '<i8'), ('upid', '<i8'), ('desc_pid', '<i8'), ('phantom', '<i8'), ('sam_mvir', '<f8'), ('mvir', '<f8'), ('rvir', '<f8'), ('rs', '<i8'), ('vrms', '<f8'), ('mmp', '<f8'), ('aexp_last_MM', '<f8'), ('vmax', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<i8'), ('jx', '<i8'), ('jy', '<i8'), ('jz', '<f8'), ('spin', '<i8'), ('b_id', '<i8'), ('d_id', '<i8'), ('tree_root_id', '<i8'), ('Orig_halo_id', '<i8'), ('nout', '<i8'), ('next_coprogenitor_d_id', '<i8'), ('last_progenitor_d_id', '<f8')])

In [2]:
sum(trees.data['nprog'] > 1)

342

In [14]:
import matplotlib.pyplot as plt
plt.hist(trees.data['nprog'])
plt.show()

In [9]:
prgs, inds = treeutils.get_main_prg(trees, haloinds=[11])
import matplotlib.pyplot as plt
#%%
import tree.treeplots as trp
a = trees.data[inds]
trp.plot_all(a, a['id'][0], save=True, out_dir=wdir)

TypeError: object of type 'NoneType' has no len()